In [2]:
import pandas as pd
import numpy as np
import re
import spacy
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import string
import joblib
import function_library as fl
import math
import plotly.express as px
import pickle
import dash
import plotly.express as px
import json
from gensim.models import Word2Vec
from dash import dcc, html, Input, Output
from datetime import datetime
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from spacy.lang.pt.stop_words import STOP_WORDS
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

In [3]:
pd.set_option('display.max_colwidth', 1000)

In [4]:
all_cus_go = pd.read_csv('F:\\project_csv\\ALL_PL_2022.csv', sep=';')

In [5]:
email_encoder = LabelEncoder()

all_cus_go['Requester Email'] = email_encoder.fit_transform(all_cus_go['Requester Email'])

In [6]:
all_cus_go.head()

,Ticket Id,Subject,Description,Source,Requester Email,Created Time,Agent interactions,Customer interactions,Tags
0,111345,[DE] [PYTANIE] B07R42W8J9,Klient ma pytanie do produktu B07R42W8J9: &quot;Jakie połączenie ma dachówka wentylacyjna?&quot; Link do zapytania: www.amazon.de/ask/question/Tx21NZNJ8PMI5JH?_encoding=UTF8&amp;authToken=967c969c-3f2e-47dd-b5e6-faeb39d052d1&amp;ref_=s_respond&amp;answerSolicitationSourceType=EMAIL&amp;answerSolicitationLanguage=de_DE Pozdrawiam Beata,Outbound Email,420,10 01 2022 10:47,4,4,"Partner_wysłane,rynek_de"
1,128291,[DE] 302-0014409-7188317 NIEKOMPLETNE,"Klient zgłasza, że w zamówieniu nie było lampki. Pozdrawiam, Jakub",Outbound Email,235,30 01 2022 09:48,3,2,"Partner_wysłane,rynek_de"
2,128325,[REKLAMACJA] 205-9594094-3558723 UK,"Dzień dobry, Klient napisał: Ten produkt dotarł, ale nie działa, nie świeci się. Bardzo rozczarowany, ponieważ kupiłem go dla mojego dziecka. Brak zauważalnych wad, wszystko wydawało się w porządku po otwarciu. Mały przycisk przesuwa się w dół i wydaje się OK, ale nic się nie dzieje, gdy go nacisnąć. To nie świeci się, co było cały punkt z nim, i bardzo rozczarowujące dla mojej córki. Pozdrawiam, Hania",Outbound Email,395,30 01 2022 10:38,2,2,"Partner_wysłane,rynek_uk,Przypomnienie"
3,131092,[DE] [ZAPYTANIE] 305-6969626-5165121,"Klient chciałby wiedzieć, czy materiał, z którego wykonane są klamki może być stosowany również na zewnątrz, np. na bocznych drzwiach wejściowych. Pozdrawiam Beata",Outbound Email,279,2 02 2022 16:10,4,4,"Partner_wysłane,rynek_de,Przypomnienie"
4,133983,[IT] 408-8392161-3066735 ZWROT,"https://www.dhl.com/it-en/home/tracking/tracking-parcel.html?submit=1&amp;tracking-id=CM332431606DE Paczka wraca do was, prosimy o refund dla klienta jak dojdzie Kasia",Outbound Email,279,7 02 2022 08:58,9,6,"Partner_wysłane,rynek_it,Przypomnienie1,Przypomnienie2"


1. Data cleaning, Feature engineering

In [7]:
format_date = "%d %m %Y %H:%M"

all_cus_go['Created Time'] = [datetime.strptime(row, format_date) for row in all_cus_go['Created Time']] 
all_cus_go['Subject'] = [str(row).lower() for row in all_cus_go['Subject']]
all_cus_go['Tags'] = [fl.split_string_by_char(row, ',')for row in all_cus_go['Tags']]
all_cus_go['rynek'] = [fl.extract_string_containing_word(row, 'rynek') for row in all_cus_go['Tags']]
all_cus_go['Order_number'] = [fl.extract_order_number(row1, row2, r"\d{3}-\d{7}-\d{7}") for row1, row2 in zip(all_cus_go['Subject'], all_cus_go['Description'])]
all_cus_go['title_list'] = [fl.split_string_by_char(row, ' ') for row in all_cus_go['Subject']]
all_cus_go['label'] = [fl.find_words_with_more_than_four_characters(row) for row in all_cus_go['Subject']]

In [8]:
all_cus_go.head()

,Ticket Id,Subject,Description,Source,Requester Email,Created Time,Agent interactions,Customer interactions,Tags,rynek,Order_number,title_list,label
0,111345,[de] [pytanie] b07r42w8j9,Klient ma pytanie do produktu B07R42W8J9: &quot;Jakie połączenie ma dachówka wentylacyjna?&quot; Link do zapytania: www.amazon.de/ask/question/Tx21NZNJ8PMI5JH?_encoding=UTF8&amp;authToken=967c969c-3f2e-47dd-b5e6-faeb39d052d1&amp;ref_=s_respond&amp;answerSolicitationSourceType=EMAIL&amp;answerSolicitationLanguage=de_DE Pozdrawiam Beata,Outbound Email,420,2022-01-10 10:47:00,4,4,"[Partner_wysłane, rynek_de]",rynek_de,,"[[de], [pytanie], b07r42w8j9]",[pytanie]
1,128291,[de] 302-0014409-7188317 niekompletne,"Klient zgłasza, że w zamówieniu nie było lampki. Pozdrawiam, Jakub",Outbound Email,235,2022-01-30 09:48:00,3,2,"[Partner_wysłane, rynek_de]",rynek_de,['302-0014409-7188317'],"[[de], 302-0014409-7188317, niekompletne]",[niekompletne]
2,128325,[reklamacja] 205-9594094-3558723 uk,"Dzień dobry, Klient napisał: Ten produkt dotarł, ale nie działa, nie świeci się. Bardzo rozczarowany, ponieważ kupiłem go dla mojego dziecka. Brak zauważalnych wad, wszystko wydawało się w porządku po otwarciu. Mały przycisk przesuwa się w dół i wydaje się OK, ale nic się nie dzieje, gdy go nacisnąć. To nie świeci się, co było cały punkt z nim, i bardzo rozczarowujące dla mojej córki. Pozdrawiam, Hania",Outbound Email,395,2022-01-30 10:38:00,2,2,"[Partner_wysłane, rynek_uk, Przypomnienie]",rynek_uk,['205-9594094-3558723'],"[[reklamacja], 205-9594094-3558723, uk]",[reklamacja]
3,131092,[de] [zapytanie] 305-6969626-5165121,"Klient chciałby wiedzieć, czy materiał, z którego wykonane są klamki może być stosowany również na zewnątrz, np. na bocznych drzwiach wejściowych. Pozdrawiam Beata",Outbound Email,279,2022-02-02 16:10:00,4,4,"[Partner_wysłane, rynek_de, Przypomnienie]",rynek_de,['305-6969626-5165121'],"[[de], [zapytanie], 305-6969626-5165121]",[zapytanie]
4,133983,[it] 408-8392161-3066735 zwrot,"https://www.dhl.com/it-en/home/tracking/tracking-parcel.html?submit=1&amp;tracking-id=CM332431606DE Paczka wraca do was, prosimy o refund dla klienta jak dojdzie Kasia",Outbound Email,279,2022-02-07 08:58:00,9,6,"[Partner_wysłane, rynek_it, Przypomnienie1, Przypomnienie2]",rynek_it,['408-8392161-3066735'],"[[it], 408-8392161-3066735, zwrot]",[zwrot]


In [9]:
rynek_encoder = LabelEncoder()

all_cus_go['rynek'] = all_cus_go['rynek'].str.replace('rynek_', '').str.replace('rynek ', '').str.replace('usa','us')
all_cus_go['rynek'] = rynek_encoder.fit_transform(all_cus_go['rynek']) + 1

In [10]:
unique_values = all_cus_go['label'].explode().tolist()
unique_values = [item for item in unique_values if not (isinstance(item, float) and math.isnan(item))]
uniq_vales, uniq_values_count = fl.list_of_unique_values_and_count_of_values(unique_values)
uniq_vales = [str(item) for item in uniq_vales]

In [11]:
nlp = spacy.load("pl_core_news_lg")

uniq_vales_sentence = ' '.join(uniq_vales)

doc = nlp(uniq_vales_sentence)

all_lem = []
all_words = []

for token in doc:
    all_words.append((token.text, token.lemma_))
    all_lem.append(token.lemma_)

dict_of_all_lemmas = {}
for lemma in set(all_lem):
    dict_of_all_lemmas[lemma] = fl.find_all_words_matching_lemma(lemma,all_words)

number_of_occurence = {}
for lemma_key in dict_of_all_lemmas:
    number_of_occurence_value = []
    for word in dict_of_all_lemmas[lemma_key]:
        number_of_occurence_value.append(uniq_values_count[word])
    number_of_occurence[lemma_key] = sum(number_of_occurence_value)

number_of_occurence_sorted = sorted(number_of_occurence.items(), key=lambda x:x[1], reverse=True)

In [12]:
label_regex ={"anulacja" : r'^[a-z]?anu?a?l|^[a-z]?nieanu?a?l',
            "reklamacja" :r'^rek|\w*k{1,}l{1,}a{1,}m{1,}a{1,}\w*',
            "wysyłka" : r'^[a-z]?wysy[łlk]|^[a-z]?niewysy?[łlk]',
            "refund" : r'^[a-z]?r[a-z]?efu',
            "przesyłka" : r'\w*esy[łl]\w*',
            "dostawa" : r'\w*dost[arw]',
            "opóźnienie" : r'^op[oó][zźżx]',
            "uszkodzony" : r'^uszko|\w*kodz\w*',
            "adres" : r'^[a-z]?ad{,2}res|^zaadre|^[a-z]?adr{,2}es',
            "ponowny" : r'^[a-z]?po{0,2}[a-z]?no',
            "paczka" : r'^paczk',
            "pytanie" : r'\w*yta\w*|^zap[yt]',
            "zwrot" : r'^zwr|\w*wrot\w*',
            "brak" : r'^[a-z]?brak',
            "niekompletny" : r'^niekom[ple]{1,}[ple]{1,}|\W*komopl',
            "faktura" : r'^fak',
            "inne" : r'^in[nm]|^i[nm]n'
}


In [13]:
dict_of_labels_and_matching_lemma_keys = {}

for label in label_regex:
    dict_of_labels_and_matching_lemma_keys[label] = []
    pattern = re.compile(label_regex[label])
    for label_key in dict_of_all_lemmas:
        if re.search(pattern, label_key):
            dict_of_labels_and_matching_lemma_keys[label].append(label_key)

not_working_keys, final_dict = fl.match_words_to_labels(dict_of_labels_and_matching_lemma_keys, dict_of_all_lemmas)
len(not_working_keys)

0

In [14]:
all_cus_go['final_label'] = [fl.find_label_in_list_of_words_from_title(final_dict, row) for row in all_cus_go['label']]
labels_distribution = pd.DataFrame(all_cus_go['final_label'].value_counts())
total_count = sum(labels_distribution['final_label'])
labels_distribution['final_label_%'] = [round(count/total_count*100,2) for count in labels_distribution['final_label']]
labels_distribution['final_label_%_cumulative'] = labels_distribution['final_label_%'].cumsum()
labels_distribution = labels_distribution.reset_index()
labels_distribution = labels_distribution.rename(columns={'final_label': 'label_count', 'index' : 'labels'})
labels_distribution.reset_index(drop=True, inplace=True)
labels_distribution

,labels,label_count,final_label_%,final_label_%_cumulative
0,dostawa,8193,22.60,22.60
1,zwrot,5878,16.21,38.81
2,anulacja,5421,14.95,53.76
3,reklamacja,4713,13.00,66.76
4,pytanie,2657,7.33,74.09
5,faktura,1859,5.13,79.22
6,wysyłka,1306,3.60,82.82
7,paczka,1255,3.46,86.28
8,adres,963,2.66,88.94
9,uszkodzony,961,2.65,91.59


2. Exploratory data analysis 

In [15]:
sunburst_df = pd.DataFrame(columns=['ids','label','parent','val', 'val_perc'])

root = {'ids' : 'labels_distribution','label' : 'labels_distribution','parent' : '','val' : all_cus_go['final_label'].count(), 'val_perc' : 100}
sunburst_df = sunburst_df.append(root, ignore_index=True) 

grouped_dict = labels_distribution[['labels', 'label_count']]
grouped_dict.set_index('labels', inplace=True)
grouped_dict = grouped_dict['label_count'].to_dict()

grouped = pd.DataFrame(all_cus_go['final_label'].value_counts())
grouped = grouped.reset_index()
grouped['ids'] = grouped['index']
grouped = grouped.rename(columns={'index' : "label", 'final_label' : 'val'})
grouped['parent'] = "labels_distribution"
grouped['val_perc'] = [fl.count_percentage_value_from_group(grouped_dict, [row1, row2]) for row1, row2 in zip(grouped['label'], grouped['val'])]
sunburst_df = pd.concat([sunburst_df, grouped], ignore_index=True)

grouped = all_cus_go.groupby(['final_label', 'rynek']).size().reset_index(name='val')
grouped['ids'] = [f"{final_label} - {rynek}" for rynek, final_label in zip(grouped["rynek"], grouped["final_label"])]
grouped = grouped.rename(columns={'rynek' : "label", 'final_label' : 'parent'})
grouped['val_perc'] = [fl.count_percentage_value_from_group(grouped_dict, [row1, row2]) for row1, row2 in zip(grouped['parent'], grouped['val'])]
sunburst_df = pd.concat([sunburst_df, grouped])

In [16]:
fig =go.Figure(go.Sunburst(
    ids = sunburst_df.ids,
    labels = sunburst_df.label,
    parents = sunburst_df.parent,
    values = sunburst_df.val_perc,
    branchvalues="total",
    insidetextorientation="radial",
))

fig.update_layout(
    title=dict(text="Labels distribution by country number", font=dict(size=30), automargin=True, yref='paper'),
    margin = dict(t=10, l=10, r=10, b=10),
    autosize=True,
    width=1000,
    height=1000,
)

fig.show()

On the label distribution plot, it is evident that over 50% of messages are distributed among three labels, and over 75% of messages are distributed among six labels. The majority of messages are directed to countries numbered 5, 7, and 8. Utilizing a model that covers these six major labels will likely result in the labeling of most incoming messages. It is recommended for future analysis to determine if any of major label messeges can be answered or partially addressed without the intervention of employees.

In [17]:
time_analys = pd.DataFrame(all_cus_go['Created Time'])
time_analys['day_of_month'] = time_analys['Created Time'].dt.day
time_analys['hours'] = time_analys['Created Time'].dt.hour
time_analys['working_hours'] = ["Working hours" if 8 <= row < 16 else "Not working hours" for row in time_analys['hours']]

df_time_analys = time_analys.groupby(['day_of_month', 'working_hours']).size().reset_index(name='val')
df_time_analys.sort_values(['day_of_month', 'working_hours'],  inplace=True)

time_analys_grouped = df_time_analys.groupby('day_of_month').sum()
time_analys_dict = time_analys_grouped['val'].to_dict()

df_time_analys['perc_val'] = [round(fl.count_percentage_value_for_label(time_analys_dict, [row1, row2]),2) for row1, row2 in zip(df_time_analys['day_of_month'], df_time_analys['val'])]

In [18]:
fig = px.bar(df_time_analys, x='day_of_month', y="perc_val", 
             color='working_hours', text="working_hours", text_auto='.3s',
             labels={'day_of_month':'Day of month', 'perc_val': '% of messages per day', "working_hours": "Hours"},
             title="Messages by day of month and working hours")

fig.show()

In [19]:
time_analys = pd.DataFrame(all_cus_go['Created Time'])
time_analys['day_of_week'] = time_analys['Created Time'].dt.dayofweek
time_analys['day_name'] = time_analys['Created Time'].dt.strftime('%A')
time_analys['hours'] = time_analys['Created Time'].dt.hour
time_analys['hours_group'] = pd.cut(time_analys['hours'], bins=3, labels=['0-7','8-15','16-24'])

df_time_analys = time_analys.groupby(['day_name', 'day_of_week', 'hours_group']).size().reset_index(name='val')
df_time_analys.sort_values(['day_of_week', 'hours_group'],  inplace=True)
df_time_analys = df_time_analys[df_time_analys['val'] != 0]

In [20]:
fig = px.bar(df_time_analys, x="day_name", y="val", color="hours_group", text="hours_group",text_auto=True,
             labels={'day_name':'Day', 'val': 'Number of messages per day', "hours_group": "Hours"},
             title="Messages by day of week and hours")

fig.show()

The plotted distribution of labels by days reveals that approximately 25% to 33% of incoming messages occur outside regular working hours. Remarkably, the message influx remains consistent, demonstrating minimal dependency on the day of the week. However, during weekends, there is an uptick to nearly 50% of the usual volume observed on regular working days. It's recommended to conduct further analysis specifically on weekend messages, focusing on identifying fields that could benefit from automation.

In [21]:
interactions = pd.DataFrame(all_cus_go[['Agent interactions', 'final_label']])

high_volume = ['dostawa', 'zwrot', 'anulacja', 'reklamacja']
medium_volume = ['pytanie', 'faktura', 'wysyłka', 'paczka']

interactions['volume'] = ['High volume' if row in high_volume else 'Medium volume' if row in medium_volume else 'Low volume' for row in interactions['final_label']]

upper_bound = interactions['Agent interactions'].quantile(0.975)

interactions_filtered = interactions[
    (interactions['Agent interactions'] <= upper_bound)
]

fig = px.box(interactions_filtered, x='final_label', y='Agent interactions',
    points='suspectedoutliers', color='volume',
    labels={'final_label':'Label', 'volume' : 'Messages Volume'},
    title="Agents interactions by labels and messages volume"
)
fig.show()

Upon reviewing agent interactions plots, it's evident that nine out of seventeen labels exhibit a count ranging from one to two interactions. The organization should prioritize investigating the 'reklamacja' and 'zwrot' message groups within the high-volume category to understand the reason behind the heightened interaction volume. Simultaneously, it's crucial to explore avenues to minimize the number of interactions within these groups.

Moreover, the 'Dostawa' and 'anulacja' groups merit thorough examination for potential automation possibilities. Should there be success in automating either the 'Dostawa' or 'anulacja' groups, a similar strategy should be implemented for the medium-volume category, where each label experiences interaction counts ranging between one to two.

Description preprocess for modeling 

In [22]:
shortcuts = []
shortcuts_dict = {}
shortcuts_pattern_dict = []

file_path = 'C:\\Users\\48575\\email_classification\\shortcuts.txt'

with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        shortcuts.append(line.replace('\xa0','').replace('\n','').split(sep='-'))

for shortcut in shortcuts:
    shortcuts_dict[shortcut[0]] = shortcut[1].strip()

for key in shortcuts_dict:
    shortcuts_pattern_dict.append(f'{{"label": "{shortcuts_dict[key]}", "pattern": [{{"TEXT": "{key}"}}]}}')

shortcuts_pattern_dict = [json.loads(item) for item in shortcuts_pattern_dict]

In [23]:
polish_names = []

file_path = 'C:\\Users\\48575\\email_classification\\imiona_polskie.txt'

with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        polish_names.append({"label" : "persName", "pattern" : [{"TEXT": line.lower().strip()}]})

In [24]:
patterns = [
    {"label" : "number", "pattern" : [{"LIKE_NUM" : True}]},
    {"label" : "web_site", "pattern" : [{"LIKE_URL" : True}]},
    {"label" : "email", "pattern" : [{"LIKE_EMAIL" : True}]},
    {"label" : "ASIN", "pattern" : [{"TEXT" : {"REGEX" : "(?i)b0[a-z0-9]{8}$"}}]}
]
patterns.extend(polish_names)

ruler = nlp.add_pipe("entity_ruler", before='ner')
ruler.add_patterns(patterns)

In [25]:
def text_preprocessing(text: str) -> str:
    """Preprocesses text by deciphering abbreviations and converting words into entities or lemmas.

    This function takes a string as an argument, changes it to an NLP object, iterates through every word, 
    and checks if it's in the shortcuts dictionary. If it finds a word in the dictionary, it retrieves its replacement value,
    processes it further by converting it to an NLP object, and iterates through resulting tokens. For each token, 
    it checks for entity types or lemmas and appends them to the processed tokens list. 
    If a word is not in the shortcuts dictionary, the function checks if it's a number, an entity type, or a lemma. 
    If it's a lemma or an entity type, it's appended to the processed tokens list. The final processed text is joined 
    back into one string.

    Args:
        text (str): String to be processed.

    Returns:
        str: Processed text with lemmas and entities.
    """
    try:
        text = text.lower().replace("&quot;", ' " ')
        processed_tokens = []

        doc = nlp(text)

        for token in doc:  
            if token.text.lower() in shortcuts_dict:
                doc2 = nlp(shortcuts_dict[token.text.lower()])
                for token2 in doc2:
                    if not token.is_stop:
                        if token2.ent_type_ != '':
                            processed_tokens.append(token2.ent_type_)
                        elif not token2.is_punct:
                            processed_tokens.append(token2.lemma_)
            elif token.is_digit:
                processed_tokens.append(token.ent_type_)
            elif not token.is_stop:
                if token.ent_type_ != '':
                    processed_tokens.append(token.ent_type_)
                elif not token.is_punct:
                    processed_tokens.append(token.lemma_)

        processed_text = " ".join(processed_tokens)

        return processed_text
    
    except:
        return text

In [26]:
# df_to_model = all_cus_go[['Subject', 'Description', 'final_label']]
# df_to_model = df_to_model[df_to_model['final_label'].notnull() & df_to_model['Description'].notnull()]
# df_to_model['desc_transformed'] = df_to_model['Description'].apply(text_preprocessing)
# df_to_model.dropna(inplace=True)
# df_to_model.to_csv('F:\\project_csv\\df_to_model.csv')

df_to_model = pd.read_csv('F:\\project_csv\\df_to_model.csv')

In [27]:
df_to_model.dropna(inplace=True)

In [28]:
top_4_labels_and_other = labels_distribution['labels'].head(4)
top_4_labels_and_other = top_4_labels_and_other.to_list()
top_4_labels_and_other

df_to_model['final_label_grouped'] = ['other' if row not in top_4_labels_and_other else row for row in df_to_model['final_label']]
df_to_model['final_label_grouped'].value_counts()

other         12055
dostawa        8193
zwrot          5876
anulacja       5414
reklamacja     4712
Name: final_label_grouped, dtype: int64

tfidf

In [29]:
X = df_to_model['desc_transformed']
y = df_to_model['final_label_grouped']

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train,
                                                    y_train,
                                                    test_size=0.2,
                                                    random_state=42)

In [31]:
tfidf = TfidfVectorizer()
tfidf.fit(X_train)

TfidfVectorizer()

In [32]:
X_train_tf = tfidf.transform(X_train)
X_test_tf = tfidf.transform(X_test)

In [33]:
X_train_tf.todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [34]:
tfidf.get_feature_names_out()

array(['00', '0000', '000w', ..., 'żądany', 'żądać', 'żę'], dtype=object)

In [35]:
pd.DataFrame(X_train_tf.todense(),
             columns=tfidf.get_feature_names_out())

,00,0000,000w,00340434390999113059,00340434410325615160,00340434463753020983,00340434660814398115,0042737,0054,0084,008b,00a,00eb5830,01,0100,01127,01169,011c,01797022158,01909,01c8a960,01d7e063,01d81b55,01d8abda,01d8b13d,01d8b600,01d8b86a,01d8b949,01d8bb9d,01d8c43d,01d8c452,01d8ccda,01d8d183,01d8d3ec,01d8d976,01d8da69,01d8dd7a,01d8e2ff,01d8e474,01d8e479,...,żandy,żartobliwie,żartujesz,żaryny,żarzyć,żarówka,żeber,żeby,żel,żelatyna,żelatyno,żelaza,żelazny,żelazo,żelowaniu,żezamówić,żeński,żle,żona,żołnierz,żołędzi,żwirek,życie,życzenie,życzyć,życzć,żyraf,żywioł,żywność,żywo,żywopłot,żółtaw,żółtawy,żółto,żółty,żółwi,żądanie,żądany,żądać,żę
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23195,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23196,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23197,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23198,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
df_functions_score = pd.DataFrame(columns=['model_name', 'label', 'score', 'metric'])


In [37]:
len(y_train)

23200

In [38]:
# lr = LogisticRegression()
# lr.fit(X_train_tf, y_train)

# joblib.dump(lr, "F:\\Classification_models\\LogisticRegression_basic")

In [39]:
lr_loaded = joblib.load("F:\\Classification_models\\LogisticRegression_basic")


y_pred = lr_loaded.predict(X_test_tf)
print(classification_report(y_test, y_pred))

df_functions_score = pd.concat([df_functions_score, fl.model_score_to_df(y_test, y_pred, "Logistic_regression_basic")], ignore_index=True)

              precision    recall  f1-score   support

    anulacja       0.98      0.95      0.96      1083
     dostawa       0.74      0.86      0.80      1617
       other       0.79      0.76      0.78      2455
  reklamacja       0.76      0.75      0.75       911
       zwrot       0.87      0.79      0.83      1184

    accuracy                           0.81      7250
   macro avg       0.83      0.82      0.82      7250
weighted avg       0.82      0.81      0.82      7250



In [40]:
# rfc = RandomForestClassifier()
# rfc.fit(X_train_tf, y_train)

# joblib.dump(rfc, "F:\\Classification_models\\RandomForestClassifier_basic")

In [41]:
rfc_loaded = joblib.load("F:\\Classification_models\\RandomForestClassifier_basic")

y_pred = rfc_loaded.predict(X_test_tf)
print(classification_report(y_test, y_pred))

df_functions_score = pd.concat([df_functions_score, fl.model_score_to_df(y_test, y_pred, "RandomForestClassifier_basic")], ignore_index=True)

              precision    recall  f1-score   support

    anulacja       0.98      0.96      0.97      1083
     dostawa       0.74      0.89      0.81      1617
       other       0.85      0.77      0.80      2455
  reklamacja       0.76      0.77      0.76       911
       zwrot       0.86      0.80      0.83      1184

    accuracy                           0.83      7250
   macro avg       0.84      0.84      0.84      7250
weighted avg       0.84      0.83      0.83      7250



In [42]:
# mnb = MultinomialNB()
# mnb.fit(X_train_tf, y_train)

# joblib.dump(mnb, "F:\\Classification_models\\mnb_basic")

In [43]:
mnb_loaded = joblib.load("F:\\Classification_models\\mnb_basic")

y_pred = mnb_loaded.predict(X_test_tf)
print(classification_report(y_test, y_pred))

df_functions_score = pd.concat([df_functions_score, fl.model_score_to_df(y_test, y_pred, "mnb_basic")], ignore_index=True)

              precision    recall  f1-score   support

    anulacja       0.99      0.92      0.95      1083
     dostawa       0.70      0.84      0.76      1617
       other       0.71      0.73      0.72      2455
  reklamacja       0.75      0.61      0.67       911
       zwrot       0.82      0.72      0.77      1184

    accuracy                           0.77      7250
   macro avg       0.79      0.76      0.78      7250
weighted avg       0.77      0.77      0.77      7250



In [44]:
# svm = SVC()
# svm.fit(X_train_tf, y_train)

# joblib.dump(svm, "F:\\Classification_models\\SVC_basic")

In [45]:
svm_loaded = joblib.load("F:\\Classification_models\\SVC_basic")

y_pred = svm_loaded.predict(X_test_tf)
print(classification_report(y_test, y_pred))

df_functions_score = pd.concat([df_functions_score, fl.model_score_to_df(y_test, y_pred, "SVC_basic")], ignore_index=True)

              precision    recall  f1-score   support

    anulacja       0.98      0.96      0.97      1083
     dostawa       0.76      0.88      0.82      1617
       other       0.83      0.77      0.80      2455
  reklamacja       0.77      0.79      0.78       911
       zwrot       0.89      0.83      0.85      1184

    accuracy                           0.84      7250
   macro avg       0.85      0.85      0.84      7250
weighted avg       0.84      0.84      0.84      7250



In [46]:
fig = px.scatter(df_functions_score, y="label", x="score", color="model_name", symbol="metric")
fig.show()

PCA

In [47]:
tfidf = TfidfVectorizer()
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

In [48]:
# pca = PCA(n_components=2000)
# X_train_pca = pca.fit_transform(X_train_tfidf.toarray())
# X_test_pca = pca.transform(X_test_tfidf.toarray())

# file_path = 'F:\\Classification_models/PCA_X_test_train.pkl'

# with open(file_path, 'wb') as f:
#     pickle.dump({'X_train_pca': X_train_pca, 'X_test_pca': X_test_pca}, f)

In [49]:
file_path = 'F:\\Classification_models/PCA_X_test_train.pkl'

with open(file_path, 'rb') as f:
    data = pickle.load(f)
    X_train_pca_loaded = data['X_train_pca']
    X_test_pca_loaded = data['X_test_pca']


In [50]:
# lr_PCA = LogisticRegression()
# lr_PCA.fit(X_train_pca_loaded, y_train)

# joblib.dump(lr_PCA, "F:\\Classification_models\\LogisticRegression_PCA")

In [51]:
lr_PCA_loaded = joblib.load("F:\\Classification_models\\LogisticRegression_PCA")


y_pred = lr_PCA_loaded.predict(X_test_pca_loaded)
print(classification_report(y_test, y_pred))

df_functions_score = pd.concat([df_functions_score, fl.model_score_to_df(y_test, y_pred, "Logistic_regression_PCA")], ignore_index=True)

              precision    recall  f1-score   support

    anulacja       0.98      0.95      0.97      1083
     dostawa       0.74      0.86      0.80      1617
       other       0.79      0.76      0.78      2455
  reklamacja       0.76      0.74      0.75       911
       zwrot       0.87      0.79      0.83      1184

    accuracy                           0.81      7250
   macro avg       0.83      0.82      0.82      7250
weighted avg       0.82      0.81      0.81      7250



In [52]:
# rfc_PCA = RandomForestClassifier()
# rfc_PCA.fit(X_train_pca_loaded, y_train)

# joblib.dump(rfc_PCA, "F:\\Classification_models\\RandomForestClassifier_PCA")

In [53]:
rfc_PCA_loaded = joblib.load("F:\\Classification_models\\RandomForestClassifier_PCA")

y_pred = rfc_PCA_loaded.predict(X_test_pca_loaded)
print(classification_report(y_test, y_pred))

df_functions_score = pd.concat([df_functions_score, fl.model_score_to_df(y_test, y_pred, "RandomForestClassifier_PCA")], ignore_index=True)

              precision    recall  f1-score   support

    anulacja       0.99      0.89      0.94      1083
     dostawa       0.72      0.76      0.74      1617
       other       0.63      0.82      0.71      2455
  reklamacja       0.75      0.42      0.54       911
       zwrot       0.85      0.63      0.72      1184

    accuracy                           0.73      7250
   macro avg       0.79      0.70      0.73      7250
weighted avg       0.75      0.73      0.73      7250



In [54]:
# svm_PCA = SVC()
# svm_PCA.fit(X_train_pca_loaded, y_train)

# joblib.dump(svm_PCA, "F:\\Classification_models\\SVC_PCA")

In [55]:
svm_PCA_loaded = joblib.load("F:\\Classification_models\\SVC_PCA")

y_pred = svm_PCA_loaded.predict(X_test_pca_loaded)
print(classification_report(y_test, y_pred))

df_functions_score = pd.concat([df_functions_score, fl.model_score_to_df(y_test, y_pred, "SVC_PCA")], ignore_index=True)

              precision    recall  f1-score   support

    anulacja       0.99      0.96      0.97      1083
     dostawa       0.76      0.89      0.82      1617
       other       0.84      0.77      0.80      2455
  reklamacja       0.77      0.80      0.78       911
       zwrot       0.88      0.83      0.85      1184

    accuracy                           0.84      7250
   macro avg       0.85      0.85      0.84      7250
weighted avg       0.84      0.84      0.84      7250



In [56]:
app = dash.Dash(__name__)

unique_labels = df_functions_score["model_name"].unique()
checkbox_options = [{'label': label, 'value': label} for label in unique_labels]

checkbox = dcc.Checklist(options=checkbox_options, value=[label for label in unique_labels], id='label-checkbox')

fig = px.scatter(df_functions_score, y="label", x="score", color="model_name", symbol="metric")

app.layout = html.Div([
    html.H1("Scatter Plot z checkboxem"),
    checkbox,
    dcc.Graph(figure=fig, id='scatter-plot')
], style={'backgroundColor': 'white'})

@app.callback(
    Output('scatter-plot', 'figure'),
    Input('label-checkbox', 'value')
)
def update_graph(selected_labels):
    filtered_data = df_functions_score[df_functions_score["model_name"].isin(selected_labels)]
    updated_fig = px.scatter(filtered_data, y="label", x="score", color="model_name", symbol="metric")
    return updated_fig

if __name__ == '__main__':
    app.run_server(debug=True)


Word2Vec - model

In [57]:
processed_emails = []

df_to_model['desc_transformed']

for mail in df_to_model['desc_transformed']:
    processed_emails.append(mail.split())

In [58]:
# word2vec_model = Word2Vec(processed_emails, vector_size=100, window=5, min_count=1, sg=1)
# word2vec_model.train(processed_emails, total_examples=len(processed_emails), epochs=100)

# joblib.dump(word2vec_model, "F:\\Classification_models\\word2vec_model")

In [59]:
word2vec_model_loaded = joblib.load("F:\\Classification_models\\word2vec_model")

In [80]:
word_count = len(word2vec_model_loaded.wv)
print("Liczba załadowanych słów:", word_count)

Liczba załadowanych słów: 29891


In [60]:
vectored_emails = []
vectored_emails_array = []

for email in processed_emails:
    email_vectors = [word2vec_model_loaded.wv[word] for word in email if word in word2vec_model_loaded.wv]
    sum_vector = np.sum(email_vectors, axis=0) if email_vectors else np.zeros(100)
    vectored_emails_array.append(sum_vector)
    vectored_emails.append(sum_vector)

vectored_emails_array = np.array(vectored_emails)

In [61]:
# len(vectored_emails)
len(vectored_emails_array)

36250

In [62]:
len(df_to_model['final_label_grouped'])

36250

In [63]:
X = vectored_emails_array
y = df_to_model['final_label_grouped']

In [64]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train,
                                                    y_train,
                                                    test_size=0.2,
                                                    random_state=42)

In [65]:
# lr = LogisticRegression()
# lr.fit(X_train, y_train)

# joblib.dump(lr, "F:\\Classification_models\\LogisticRegression_word2vec_basic")

In [66]:
lr_loaded = joblib.load("F:\\Classification_models\\LogisticRegression_word2vec_basic")


y_pred = lr_loaded.predict(X_test)
print(classification_report(y_test, y_pred))

df_functions_score = pd.concat([df_functions_score, fl.model_score_to_df(y_test, y_pred, "LogisticRegression_word2vec_basic")], ignore_index=True)

              precision    recall  f1-score   support

    anulacja       0.94      0.94      0.94      1083
     dostawa       0.69      0.83      0.76      1617
       other       0.77      0.65      0.71      2455
  reklamacja       0.71      0.67      0.69       911
       zwrot       0.75      0.79      0.77      1184

    accuracy                           0.76      7250
   macro avg       0.77      0.78      0.77      7250
weighted avg       0.76      0.76      0.76      7250



In [67]:
# rfc = RandomForestClassifier()
# rfc.fit(X_train_tf, y_train)

# joblib.dump(rfc, "F:\\Classification_models\\RandomForestClassifier_word2vec_basic")

In [68]:
rfc_loaded = joblib.load("F:\\Classification_models\\RandomForestClassifier_word2vec_basic")

y_pred = rfc_loaded.predict(X_test_tf)
print(classification_report(y_test, y_pred))

df_functions_score = pd.concat([df_functions_score, fl.model_score_to_df(y_test, y_pred, "RandomForestClassifier_word2vec_basic")], ignore_index=True)

              precision    recall  f1-score   support

    anulacja       0.98      0.96      0.97      1083
     dostawa       0.75      0.89      0.82      1617
       other       0.86      0.76      0.81      2455
  reklamacja       0.76      0.79      0.77       911
       zwrot       0.85      0.82      0.83      1184

    accuracy                           0.83      7250
   macro avg       0.84      0.84      0.84      7250
weighted avg       0.84      0.83      0.83      7250



In [69]:
# mnb = MultinomialNB()
# mnb.fit(X_train_tf, y_train)

# joblib.dump(mnb, "F:\\Classification_models\\mnb_word2vec_basic")

In [70]:
mnb_loaded = joblib.load("F:\\Classification_models\\mnb_word2vec_basic")

y_pred = mnb_loaded.predict(X_test_tf)
print(classification_report(y_test, y_pred))

df_functions_score = pd.concat([df_functions_score, fl.model_score_to_df(y_test, y_pred, "mnb_word2vec_basic")], ignore_index=True)

              precision    recall  f1-score   support

    anulacja       0.99      0.92      0.95      1083
     dostawa       0.70      0.84      0.76      1617
       other       0.71      0.73      0.72      2455
  reklamacja       0.75      0.61      0.67       911
       zwrot       0.82      0.72      0.77      1184

    accuracy                           0.77      7250
   macro avg       0.79      0.76      0.78      7250
weighted avg       0.77      0.77      0.77      7250



In [71]:
# svm = SVC()
# svm.fit(X_train_tf, y_train)

# joblib.dump(svm, "F:\\Classification_models\\SVC_word2vec_basic")

In [72]:
svm_loaded = joblib.load("F:\\Classification_models\\SVC_word2vec_basic")

y_pred = svm_loaded.predict(X_test_tf)
print(classification_report(y_test, y_pred))

df_functions_score = pd.concat([df_functions_score, fl.model_score_to_df(y_test, y_pred, "SVC_word2vec_basic")], ignore_index=True)

              precision    recall  f1-score   support

    anulacja       0.98      0.96      0.97      1083
     dostawa       0.76      0.88      0.82      1617
       other       0.83      0.77      0.80      2455
  reklamacja       0.77      0.79      0.78       911
       zwrot       0.89      0.83      0.85      1184

    accuracy                           0.84      7250
   macro avg       0.85      0.85      0.84      7250
weighted avg       0.84      0.84      0.84      7250



In [73]:
app = dash.Dash(__name__)

unique_labels = df_functions_score["model_name"].unique()
checkbox_options = [{'label': label, 'value': label} for label in unique_labels]

checkbox = dcc.Checklist(options=checkbox_options, value=[label for label in unique_labels], id='label-checkbox')

fig = px.scatter(df_functions_score, y="label", x="score", color="model_name", symbol="metric")

app.layout = html.Div([
    html.H1("Scatter Plot z checkboxem"),
    checkbox,
    dcc.Graph(figure=fig, id='scatter-plot')
], style={'backgroundColor': 'white'})

@app.callback(
    Output('scatter-plot', 'figure'),
    Input('label-checkbox', 'value')
)
def update_graph(selected_labels):
    filtered_data = df_functions_score[df_functions_score["model_name"].isin(selected_labels)]
    updated_fig = px.scatter(filtered_data, y="label", x="score", color="model_name", symbol="metric")
    return updated_fig

if __name__ == '__main__':
    app.run_server(debug=True)


In [74]:
X = vectored_emails
y = df_to_model['final_label_grouped']


In [75]:
label_encoder = LabelEncoder()

encoded_labels = label_encoder.fit_transform(y)
y_train_onehot = to_categorical(encoded_labels)

In [76]:

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y_train_onehot,
                                                    test_size=0.2,
                                                    random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train,
                                                    y_train,
                                                    test_size=0.2,
                                                    random_state=42)

In [77]:
word_count = len(word2vec_model_loaded.wv)

23200

In [78]:
import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding, Flatten


# define the model
model = Sequential()
model.add(Embedding(input_dim=word_count+1, 8, input_length=max_length))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# summarize the model
print(model.summary())

NameError: name 'vocab_size' is not defined

In [ ]:

# fit the model
model.fit(padded_docs, labels, epochs=50, verbose=0)
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

In [ ]:
i == 1

In [ ]:
# Inicjalizacja i dopasowanie modelu regresji logistycznej
model = LogisticRegression(max_iter=500)  # Ustawienie liczby iteracji
model.fit(X_train_pca_loaded, y_train)

# Predykcja na danych testowych
y_pred = model.predict(X_train_pca_loaded)

In [ ]:
nlp = spacy.blank("pl")

In [ ]:
df = pd.DataFrame(df_to_model[['Description', 'final_label']])
df.reset_index(inplace=True)

In [ ]:
REGX_URL = r"(https?://|www\.)[^\s/$.?#].[^\s]*"

def preprocessing(text):
  text = text.lower()
  text = text.replace("&quot;", '"')

  text = re.sub(REGX_URL, ' ', text)

  tokens = [token.text for token in nlp(text)]

  tokens = [t for t in tokens if 
              t not in STOP_WORDS and 
              t not in string.punctuation]

  tokens = [t for t in tokens if not t.isdigit()]

  return " ".join(tokens)

In [ ]:
df['Description_cleaned'] = df['Description'].apply(preprocessing)

In [ ]:
y =pd.get_dummies((df['final_label']), prefix='label')
label = list(y.columns)
y = y.to_dict('index')

In [ ]:
dataset = list(zip(df['Description_cleaned'], [{'cats':cats} for cats in y.values()]))
print(dataset[0])

('klient ma pytanie produktu b07r42w8j9 jakie połączenie ma dachówka wentylacyjna link zapytania    pozdrawiam beata', {'cats': {'label_adres': 0, 'label_anulacja': 0, 'label_brak': 0, 'label_dostawa': 0, 'label_faktura': 0, 'label_inne': 0, 'label_niekompletny': 0, 'label_opóźnienie': 0, 'label_paczka': 0, 'label_ponowny': 0, 'label_przesyłka': 0, 'label_pytanie': 1, 'label_refund': 0, 'label_reklamacja': 0, 'label_uszkodzony': 0, 'label_wysyłka': 0, 'label_zwrot': 0}})


In [ ]:
train_data, test_data = train_test_split(dataset,
                                        test_size=0.25,
                                        random_state=42)
dev_data, test_data = train_test_split(test_data, 
                                       test_size=0.4,
                                       random_state=42)

print(f"Total: {len(dataset)} - Train:  {len(train_data)} - Dev: {len(dev_data)} - Test: {len(test_data)}")

Total: 36234 - Train:  27175 - Dev: 5435 - Test: 3624


In [ ]:
def convert(data, outfile):
    db = spacy.tokens.DocBin()

    for text, labels in data:
        doc = spacy.tokens.Doc(nlp.vocab, words=text.split())
        doc.cats.update(labels['cats'])
        db.add(doc)

    db.to_disk(outfile)

convert(train_data, "./train.spacy")
convert(dev_data, "./dev.spacy")
convert(test_data, "./test.spacy")

In [ ]:
textcat = nlp.add_pipe("textcat_multilabel")
for i in label:
    textcat.add_label(i)

In [ ]:
# ! python -m spacy init fill-config base_config.cfg config.cfg

In [ ]:
# ! python -m spacy train config.cfg --paths.train ./train.spacy  --paths.dev ./dev.spacy --output model --verbose

In [ ]:
# output_dir = 'C:\\Users\\48575\\email_classification\\spaCy_model'
# nlp.to_disk(output_dir)

In [ ]:
saved_model_dir = 'C:\\Users\\48575\\email_classification\\spaCy_model'
nlp = spacy.load(saved_model_dir)

In [ ]:
! python -m spacy evaluate ./model/model-best/ ./test.spacy

ℹ Using CPU

================================== Results ==================================

TOK                   -     
TEXTCAT (macro AUC)   76.36 
SPEED                 163843


=========================== Textcat F (per label) ===========================

                          P       R       F
label_adres           81.25   93.81   87.08
label_anulacja        98.66   91.50   94.95
label_brak            67.39   84.93   75.15
label_dostawa         94.78   15.58   26.77
label_faktura         98.47   97.47   97.97
label_inne           100.00   62.50   76.92
label_niekompletne     0.00    0.00    0.00
label_opóźnienie     100.00   27.66   43.33
label_paczka          82.30   68.38   74.70
label_ponowna          0.00    0.00    0.00
label_przesyłka       63.83   88.24   74.07
label_refund          49.32   93.59   64.60
label_reklamacja      85.49   48.02   61.50
label_uszkodzony      78.43   45.45   57.55
label_wysyłka         78.57   16.06   26.67
label_zapytanie        0.00    0.00 

In [ ]:
def predict_spaCy (text):
    preprocessed_text = preprocessing(text)
    doc = nlp(preprocessed_text)
    label = max(doc.cats, key=doc.cats.get)
    return label

Optimization of Logistic Regression, Random Forest Classifier, and Naive Bayes

In [ ]:
X = df['Description_cleaned']
y = df['final_label']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=42)

In [ ]:
tfidf = TfidfVectorizer()
tfidf.fit(X_train)

TfidfVectorizer()

In [ ]:
X_train_tf = tfidf.transform(X_train)
X_test_tf = tfidf.transform(X_test)

In [ ]:
X_train_tf.todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:


lr = LogisticRegression()
lr.fit(X_train_tf, y_train)

y_pred = lr.predict(X_test_tf)

from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       adres       0.82      0.74      0.78       200
    anulacja       0.97      0.95      0.96      1135
        brak       0.77      0.48      0.59       164
     dostawa       0.72      0.91      0.80      1647
     faktura       1.00      0.97      0.98       379
        inne       0.85      0.52      0.64        33
niekompletny       0.89      0.46      0.60        90
  opóźnienie       0.89      0.59      0.71        83
      paczka       0.90      0.58      0.70       260
     ponowny       0.85      0.35      0.50        31
   przesyłka       0.86      0.50      0.63        72
     pytanie       0.88      0.91      0.89       526
      refund       0.93      0.65      0.77       153
  reklamacja       0.67      0.87      0.76       867
  uszkodzony       0.82      0.51      0.63       175
     wysyłka       0.75      0.14      0.23       261
       zwrot       0.85      0.83      0.84      1171

    accuracy              

In [ ]:
param_grid = {
    'penalty': ['l1','l2','None'],
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'fit_intercept': [True, False],
    'tol' : [0.001,0.01],
    'solver': ['newton-cg', 'lbfgs', 'sag', 'saga'],
    'max_iter': [100, 200],
    'multi_class': ['auto', 'multinomial'],
}

In [ ]:
# searcher=RandomizedSearchCV(lr, param_grid)
# searcher.fit(X_train_tf, y_train)

In [ ]:
# joblib.dump(searcher, "LogisticRegression_randomizedSearchCV")

In [ ]:
searcher = joblib.load("LogisticRegression_randomizedSearchCV")

In [ ]:
print(searcher.best_score_)
print(searcher.best_params_)

0.7918849153934211
{'solver': 'saga', 'penalty': 'l2', 'multi_class': 'ovr', 'fit_intercept': True, 'C': 10}


In [ ]:
lr = LogisticRegression(tol=0.01, solver='newton-cg', penalty='l2', multi_class='multinomial', 
                        max_iter=200, fit_intercept=True, C=10
                        )
lr.fit(X_train_tf, y_train)

y_pred = lr.predict(X_test_tf)


print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       adres       0.80      0.78      0.79       200
    anulacja       0.97      0.96      0.97      1135
        brak       0.76      0.66      0.71       164
     dostawa       0.75      0.90      0.82      1647
     faktura       0.99      0.98      0.99       379
        inne       0.86      0.55      0.67        33
niekompletny       0.84      0.59      0.69        90
  opóźnienie       0.88      0.69      0.77        83
      paczka       0.86      0.66      0.75       260
     ponowny       0.65      0.48      0.56        31
   przesyłka       0.75      0.67      0.71        72
     pytanie       0.89      0.91      0.90       526
      refund       0.87      0.69      0.77       153
  reklamacja       0.72      0.86      0.79       867
  uszkodzony       0.76      0.61      0.68       175
     wysyłka       0.70      0.17      0.28       261
       zwrot       0.87      0.83      0.85      1171

    accuracy              

In [ ]:

rfc = RandomForestClassifier()
rfc.fit(X_train_tf, y_train)

y_pred = rfc.predict(X_test_tf)



print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       adres       0.83      0.69      0.75       200
    anulacja       0.97      0.95      0.96      1135
        brak       0.83      0.51      0.63       164
     dostawa       0.71      0.93      0.80      1647
     faktura       0.99      0.97      0.98       379
        inne       0.86      0.55      0.67        33
niekompletny       0.85      0.37      0.51        90
  opóźnienie       0.85      0.57      0.68        83
      paczka       0.90      0.55      0.69       260
     ponowny       0.80      0.39      0.52        31
   przesyłka       0.80      0.56      0.66        72
     pytanie       0.90      0.90      0.90       526
      refund       0.91      0.67      0.77       153
  reklamacja       0.69      0.86      0.77       867
  uszkodzony       0.84      0.45      0.59       175
     wysyłka       0.84      0.15      0.25       261
       zwrot       0.84      0.83      0.84      1171

    accuracy              

In [ ]:
i == 1

In [ ]:
param_grid = {
    'n_estimators': [50, 125, 200],  # Range of values for n_estimators
    'max_depth': [None, 5, 13, 20],  # Including None for no max depth
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 4, 8],
    'max_features': ['auto', 'sqrt', 'log2', None],
    'criterion': ['gini', 'entropy']
}

In [ ]:
# searcher=RandomizedSearchCV(rfc, param_grid)
# searcher.fit(X_train_tf, y_train)

In [ ]:
# joblib.dump(searcher, "RandomForestClassifier_randomizedSearchCV")

In [ ]:
searcher = joblib.load("LogisticRegression_randomizedSearchCV")

In [ ]:
print(searcher.best_score_)
print(searcher.best_params_)

0.7918849153934211
{'solver': 'saga', 'penalty': 'l2', 'multi_class': 'ovr', 'fit_intercept': True, 'C': 10}


In [ ]:
# rfc = RandomForestClassifier(n_estimators=200, min_samples_split=2, min_samples_leaf=1, max_features=None, max_depth=None, criterion='gini')
# rfc.fit(X_train_tf, y_train)

# y_pred = rfc.predict(X_test_tf)

# from sklearn.metrics import classification_report

# print(classification_report(y_test, y_pred))